# Sheet

In [1]:
import clickhouse_connect
import sys
import json
import pyarrow as pa
import implicit
import bisect
import scipy
import numpy as np
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV


CLICKHOUSE_CLOUD_HOSTNAME = 'toxicus-vulpes.ru'
CLICKHOUSE_CLOUD_USER = 'default'
CLICKHOUSE_CLOUD_PASSWORD = ''
client = clickhouse_connect.get_client(
    host=CLICKHOUSE_CLOUD_HOSTNAME, port=18123, username=CLICKHOUSE_CLOUD_USER, password=CLICKHOUSE_CLOUD_PASSWORD)

In [2]:
df_1 = client.query_df("""
SELECT user_id, referer, SUM(request_cnt_sum) as request_cnt_sum
FROM (SELECT user_id, 
CAST(extractAll(referer, '\d{1,20}')[1] AS INT) AS referer, 
count(timestamp) as request_cnt_sum 
FROM requests 
INNER JOIN train_users tu on tu.user_id = requests.user_id
WHERE requests.referer != ''
GROUP BY referer, user_id)
GROUP BY user_id, referer
""")

In [3]:
df = df_1
# df = df.append(df_2, ignore_index = True )

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20528762 entries, 0 to 20528761
Data columns (total 3 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          int32 
 1   referer          int32 
 2   request_cnt_sum  uint64
dtypes: int32(2), uint64(1)
memory usage: 313.2 MB


In [5]:
da = pa.Table.from_pandas(df)

In [6]:
url_set = set(da.select(['referer']).to_pandas()['referer'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(da.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

3369 urls
4999877 users


In [7]:
values = np.array(da.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(da.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(da.select(['referer']).to_pandas()['referer'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.als.AlternatingLeastSquares(factors = 50)

/opt/python/envs/minimal/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [23]:
als.fit(mat)

/opt/python/envs/minimal/lib/python3.8/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.7240984439849854 seconds
  warnings.warn(


In [30]:
d_factors = als.item_factors

In [31]:
usr_targets = client.query_df("SELECT user_id, age, gender FROM default.train_users")

In [32]:
import pandas as pd
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(d_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)

In [34]:

# usr_targets = targets.to_pandas()
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['gender'] != 'NA']
df = df.dropna()
df['gender'] = df['gender'].map(int)
df['gender'].value_counts()

In [47]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
x_train, x_test, y_train, y_test = train_test_split( df.drop(['user_id', 'age', 'gender'], axis = 1), df['gender'], test_size = 0.33, random_state = 42)
clf = RandomForestClassifier()

clf.fit(x_train, y_train)
print(f'GINI по полу {2 * m.accuracy_score(y_test, clf.predict(x_test)) - 1:2.3f}')

In [48]:
id_to_submit = df['user_id']
id_to_submit = id_to_submit.to_frame()
id_to_submit.info()

In [49]:
id_ts = id_to_submit.merge(usr_emb, how = 'left', on = ['user_id'])
id_ts.columns = map(lambda x: str(x), id_ts.columns)
len(clf.predict(id_ts.drop(columns=['user_id'])))

In [50]:
id_to_submit['gender'] = clf.predict(id_ts.drop(columns=['user_id']))

In [51]:
id_to_submit

In [52]:
from sklearn.metrics import accuracy_score

accuracy_score(df['gender'], id_to_submit['gender'])

In [45]:
from lazypredict.Supervised import LazyClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelSpreading
from xgboost import XGBClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None, classifiers=[RandomForestClassifier, XGBClassifier, 
                                                                                      ExtraTreesClassifier, LabelSpreading, 
                                                                                      KNeighborsClassifier, CatBoostClassifier])
models,predictions = clf.fit(x_train, x_test, y_train, y_test)
models

In [53]:
models.to_csv("/data/notebook_files/models.csv")

In [55]:
clf.feature_importances_

# Sheet 2

In [62]:
# usr_with_gender = id_ts.merge(id_to_submit, how = 'left', on = ['user_id'])

In [81]:
x_train, x_test, y_train, y_test = train_test_split( df.drop(['user_id', 'age', 'gender'], axis = 1), df['age'], test_size = 0.33, random_state = 42)
regr = CatBoostRegressor()

regr.fit(x_train, y_train, verbose=False)

print(f'GINI по возрасту {2 * m.accuracy_score(y_test, regr.predict(x_test).round()) - 1:2.3f}')

In [82]:
id_to_submit['age'] = regr.predict(id_ts.drop(columns=['user_id'])).round()

In [83]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(df['age'], id_to_submit['age'])

In [70]:
from lazypredict.Supervised import LazyRegressor
from sklearn.linear_model import ElasticNet, BayesianRidge, LassoLarsCV, LassoLarsIC
from sklearn.svm import SVR

regr = LazyRegressor(verbose=0,ignore_warnings=True, regressors=[SVR, ElasticNet, BayesianRidge, 
                                                                 LassoLarsCV, LassoLarsIC, 
                                                                 CatBoostRegressor], custom_metric=mean_absolute_error)
models,predictions = regr.fit(x_train, x_test, y_train, y_test)
models

In [71]:
models.to_csv("/data/notebook_files/models.csv")

# Sheet 3

In [85]:
client.insert_df('test_users', pd.read_csv("/data/notebook_files/test_users.csv"))

In [86]:
select * from test_users

# Sheet 4

In [88]:
df_1 = client.query_df("""
SELECT user_id, referer, SUM(request_cnt_sum) as request_cnt_sum
FROM (SELECT user_id, 
CAST(extractAll(referer, '\d{1,20}')[1] AS INT) AS referer, 
count(timestamp) as request_cnt_sum 
FROM requests 
INNER JOIN test_users tu on tu.user_id = requests.user_id
WHERE requests.referer != ''
GROUP BY referer, user_id)
GROUP BY user_id, referer
""")
df = df_1

In [89]:
da = pa.Table.from_pandas(df)
url_set = set(da.select(['referer']).to_pandas()['referer'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(da.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

In [90]:
values = np.array(da.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(da.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(da.select(['referer']).to_pandas()['referer'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.als.AlternatingLeastSquares(factors = 50)

In [91]:
als.fit(mat)

In [92]:
d_factors = als.item_factors

In [93]:
usr_targets = client.query_df("SELECT user_id FROM default.test_users")

In [94]:
import pandas as pd
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(d_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)

In [ ]:
id_to_submit['gender'] = clf.predict(usr_emb.drop(columns=['user_id']))
id_to_submit['age'] = regr.predict(usr_emb.drop(columns=['user_id'])).round()